<a href="https://colab.research.google.com/github/isuru0x01/Notebooks/blob/main/Modular_CV_Framework_with_Extensible_Layer_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [3]:
class ConvLayerModule:
    def __init__(self, filters, kernel_size, dilation_rate=1):
        self.filters = filters
        self.kernel_size = kernel_size
        self.dilation_rate = dilation_rate

    def build_layer(self):
        return layers.Conv2D(filters=self.filters,
                             kernel_size=self.kernel_size,
                             dilation_rate=self.dilation_rate,
                             activation='relu')

In [4]:
class AttentionLayerModule:
    def __init__(self, units):
        self.units = units

    def build_layer(self):
        return layers.Attention()

In [5]:
class SequenceModelingModule:
    def __init__(self, input_shape, conv_layer_module, attention_layer_module=None):
        self.input_shape = input_shape
        self.conv_layer_module = conv_layer_module
        self.attention_layer_module = attention_layer_module
        self.model = self.build_model()

    def build_model(self):
        inputs = layers.Input(shape=self.input_shape)
        x = self.conv_layer_module.build_layer()(inputs)
        if self.attention_layer_module:
            x = self.attention_layer_module.build_layer()([x, x])
        x = layers.Flatten()(x)
        x = layers.Dense(64, activation='relu')(x)
        outputs = layers.Dense(10, activation='softmax')(x)
        model = models.Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

    def train(self, X, y, epochs=3):
        self.model.fit(X, y, epochs=epochs)

    def predict(self, X):
        return self.model.predict(X)


In [6]:
class CVFramework:
    def __init__(self, input_shape, conv_layer_params, attention_layer_params=None):
        self.conv_layer_module = ConvLayerModule(**conv_layer_params)
        self.attention_layer_module = AttentionLayerModule(**attention_layer_params) if attention_layer_params else None
        self.sequence_model = SequenceModelingModule(input_shape, self.conv_layer_module, self.attention_layer_module)

    def train(self, X, y, epochs=3):
        self.sequence_model.train(X, y, epochs)

    def predict(self, X):
        return self.sequence_model.predict(X)


In [7]:
# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize the data
X_train, X_test = X_train / 255.0, X_test / 255.0

# Model parameters
input_shape = X_train.shape[1:]
conv_layer_params = {'filters': 32, 'kernel_size': 3, 'dilation_rate': 1}
attention_layer_params = {'units': 64}  # Only needed if using attention

# Initialize and train the framework with standard convolution
framework = CVFramework(input_shape, conv_layer_params)
framework.train(X_train, y_train, epochs=3)

# Predict on test data
predictions = framework.predict(X_test)
print(predictions[:5])  # Print first 5 predictions


170498071/170498071 [==============================] - 3s 0us/step
Epoch 1/3
1563/1563 [==============================] - 63s 40ms/step - loss: 1.6657 - accuracy: 0.3991
Epoch 2/3
1563/1563 [==============================] - 61s 39ms/step - loss: 1.3377 - accuracy: 0.5200
Epoch 3/3
313/313 [==============================] - 3s 10ms/step
[[9.25594047e-02 3.16347834e-03 1.10440984e-01 4.32891786e-01
  1.62472334e-02 2.49033064e-01 2.70461719e-02 1.28147705e-03
  6.52308986e-02 2.10541673e-03]
 [3.05623449e-02 4.07679677e-01 3.13984060e-06 1.78843266e-07
  3.74643534e-08 1.91633713e-08 1.51886567e-07 4.22123847e-08
  5.48676670e-01 1.30777126e-02]
 [2.44830072e-01 1.08403407e-01 2.77929427e-03 1.00258668e-03
  1.29533990e-04 5.01256029e-04 2.99756703e-05 5.49657794e-04
  5.35028398e-01 1.06745869e-01]
 [6.00766838e-01 2.95718592e-02 5.26173925e-03 1.95517088e-04
  7.03622936e-04 5.20656649e-05 7.71878113e-05 2.03899559e-04
  3.58437449e-01 4.72991634e-03]
 [7.86805409e-04 8.13140301e-04 1

In [8]:
# Change to dilated convolutions
conv_layer_params['dilation_rate'] = 2  # Example dilation rate

# Reinitialize and train the framework with dilated convolutions
framework = CVFramework(input_shape, conv_layer_params)
framework.train(X_train, y_train, epochs=3)

# Predict on test data
predictions = framework.predict(X_test)
print(predictions[:5])  # Print first 5 predictions


Epoch 1/3
1563/1563 [==============================] - 58s 36ms/step - loss: 1.5510 - accuracy: 0.4478
Epoch 2/3
1563/1563 [==============================] - 58s 37ms/step - loss: 1.3022 - accuracy: 0.5395
Epoch 3/3
313/313 [==============================] - 3s 10ms/step
[[5.57674095e-02 4.38387226e-03 1.84823945e-01 3.79323989e-01
  2.91444603e-02 1.42243162e-01 1.48666352e-01 3.74078657e-03
  4.71707620e-02 4.73511731e-03]
 [7.90235251e-02 2.66711324e-01 2.12209648e-04 1.46034590e-05
  1.60420444e-04 1.68033091e-06 7.71801660e-06 2.95385635e-05
  5.47801375e-01 1.06037594e-01]
 [2.88312554e-01 1.16693549e-01 3.12486454e-03 4.42992756e-03
  5.08762198e-04 1.69050088e-03 1.90415463e-04 7.75742286e-04
  5.09718716e-01 7.45549649e-02]
 [5.03964484e-01 1.07006589e-02 4.97785863e-03 3.45252862e-04
  1.39726768e-03 6.89098597e-05 2.00205948e-04 8.59490887e-04
  4.72801715e-01 4.68415627e-03]
 [3.15971934e-04 7.50674866e-04 6.37589768e-02 1.43062502e-01
  6.87199950e-01 5.71228787e-02 4.4520

In [9]:
# Initialize and train the framework with attention mechanism
framework = CVFramework(input_shape, conv_layer_params, attention_layer_params)
framework.train(X_train, y_train, epochs=3)

# Predict on test data
predictions = framework.predict(X_test)
print(predictions[:5])  # Print first 5 predictions


Epoch 1/3
1563/1563 [==============================] - 103s 66ms/step - loss: 1.6597 - accuracy: 0.3956
Epoch 2/3
1563/1563 [==============================] - 101s 65ms/step - loss: 1.4420 - accuracy: 0.4822
Epoch 3/3
313/313 [==============================] - 6s 20ms/step
[[7.5827576e-03 3.5958274e-03 3.4017373e-02 5.6194776e-01 3.1993199e-02
  3.1832707e-01 2.5065443e-02 6.4562145e-03 8.9282151e-03 2.0861158e-03]
 [3.4886044e-01 1.5902543e-01 2.2482928e-03 7.9760604e-05 5.6787656e-04
  4.9971779e-05 2.1624650e-05 1.3934210e-03 4.5387363e-01 3.3879530e-02]
 [1.4675480e-01 5.0400335e-02 5.8789249e-03 2.0529837e-03 1.9714339e-03
  1.1622502e-03 1.6088810e-04 2.1132706e-03 7.7152711e-01 1.7977975e-02]
 [2.8950590e-01 2.8954523e-02 1.2093485e-02 2.4156605e-03 1.0640982e-02
  1.6772717e-03 8.4397732e-05 4.8913443e-03 6.3921952e-01 1.0516980e-02]
 [1.5770404e-02 6.9978805e-03 1.0451374e-01 2.7339235e-01 2.8571004e-01
  1.0060526e-01 1.8277413e-01 2.0247290e-02 3.0315276e-03 6.9574006e-03]]
